In [1]:
!nvidia-smi

Thu Dec 19 04:48:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4000               On  | 00000000:01:00.0 Off |                  Off |
| 41%   31C    P8              16W / 140W |     27MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install yt-dlp qwen-vl-utils flash-attn --no-build-isolation accelerate
# Clean up the notebook
clean_notebook()



Notebook cleaned.


In [3]:
from yt_dlp import YoutubeDL

YOUTUBE_URL = "https://www.youtube.com/watch?v=MCWJNOfJoSM"

ydl_opts = {
    "format": "best",
    "outtmpl":"football.mp4"
    }
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

[youtube] Extracting URL: https://www.youtube.com/watch?v=MCWJNOfJoSM
[youtube] MCWJNOfJoSM: Downloading webpage
[youtube] MCWJNOfJoSM: Downloading ios player API JSON
[youtube] MCWJNOfJoSM: Downloading mweb player API JSON
[youtube] MCWJNOfJoSM: Downloading m3u8 information
[info] MCWJNOfJoSM: Downloading 1 format(s): 18
[download] Destination: football.mp4
[download] 100% of   39.07MiB in 00:00:00 at 45.74MiB/s    


In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
import torch

import torch

MODEL_NAME = "Qwen/Qwen2-VL-7B-Instruct"

# Define the quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Change to `load_in_8bit=True` for 8-bit quantization
    bnb_4bit_use_double_quant=True,  # Use double quantization for better accuracy
    bnb_4bit_quant_type="nf4",       # NormalFloat4 (nf4) quantization type
    bnb_4bit_compute_dtype=torch.float16  # Compute in float16 for lower memory
)

# Load the model with quantization
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,  # Pass quantization config
    device_map="auto",
    #attn_implementation="flash_attention_2",
)

# Load the processor
processor = AutoProcessor.from_pretrained(MODEL_NAME)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
# from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
# from qwen_vl_utils import process_vision_info
# import torch

# MODEL_NAME = "Qwen/Qwen2-VL-7B-Instruct"

# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     attn_implementation="flash_attention_2",
# )

# processor = AutoProcessor.from_pretrained(MODEL_NAME)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "football.mp4",
                "max_pixels": 1280 * 780,
                "fps": 0.1,
            },
            {"type": "text", "text": "What's happening in the video? who wins the penalty shootout?."},
        ],
    }
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=512)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text)

qwen-vl-utils using torchvision to read video.
